In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
# Parameters
N = 2
a = 1.0
c = -0.2

potential = 'quadratic'
boundary_condition = 'periodic'
cutoff = 4

In [3]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\VQE\DE\Optimizing\VaryPopsize\periodic\quadratic\C0.2\N2"
dataframes = {}
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders

['2025-02-11_09-57-00',
 '2025-02-11_09-57-14',
 '2025-02-11_09-58-02',
 '2025-02-11_09-58-36',
 '2025-02-11_10-14-29',
 '2025-02-11_10-34-02',
 '2025-02-11_10-34-56']

In [4]:
converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []

Popsizes = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    Popsizes.append(data['Optimizer']['popsize'])

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])
    abs_tolerance.append(optimzer['abs_tolerance'])
    strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['parallel_run_time'])

    

In [5]:
df = pd.DataFrame({
    'folder': Popsizes,
    'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    'abs_tolerance': abs_tolerance,
    'strategy': strategy,
    'popsize': popsize,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [6]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,folder,num_vqe,shots,tolerance,abs_tolerance,strategy,popsize,Converged Runs,maxiter,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
4,5,40,1024,0.001,0.001,rand1bin,5,16/40,10000,7218,4.11547e-01,5.33728e-03,4.67319e-01,6.11094e-02,4.06210e-01,0:05:38.808923
1,15,40,1024,0.001,0.001,rand1bin,15,33/40,10000,5992,4.06210e-01,1.77636e-15,4.06210e-01,2.10942e-15,4.06210e-01,0:15:15.729402
2,20,40,1024,0.001,0.001,rand1bin,20,35/40,10000,5977,4.06210e-01,1.77636e-15,4.06210e-01,2.10942e-15,4.06210e-01,0:20:18.298385
6,35,40,1024,0.001,0.001,rand1bin,35,25/40,10000,9053,4.06210e-01,1.77636e-15,4.06210e-01,2.10942e-15,4.06210e-01,0:40:44.782141
0,10,40,1024,0.001,0.001,rand1bin,10,27/40,10000,5498,4.06210e-01,1.77636e-15,4.07544e-01,1.33432e-03,4.06210e-01,0:09:56.837361
5,25,40,1024,0.001,0.001,rand1bin,25,38/40,10000,6850,4.06210e-01,1.72085e-15,4.06210e-01,2.10942e-15,4.06210e-01,0:26:12.844177
3,30,40,1024,0.001,0.001,rand1bin,30,28/40,10000,7784,4.06210e-01,1.66533e-15,4.06210e-01,2.10942e-15,4.06210e-01,0:33:11.263372
